## Final Project Submission

Please fill out:
* Student name: Harrison Gu and Justin Hue
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Yish Lim

Business question: We are running a construction company and would like to determine how to build houses that sell at a higher price.


Import necessary libraries

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from itertools import combinations

Investigate the data

In [2]:
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

Check to see if any houses appear more than once. If so, remove duplicates.

In [4]:
df[df.duplicated(subset='id', keep=False)==True]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
93,6021501535,7/25/2014,430000.0,3,1.50,1580,5000,1.0,0.0,0.0,...,8,1290,290.0,1939,0.0,98117,47.6870,-122.386,1570,4500
94,6021501535,12/23/2014,700000.0,3,1.50,1580,5000,1.0,0.0,0.0,...,8,1290,290.0,1939,0.0,98117,47.6870,-122.386,1570,4500
313,4139480200,6/18/2014,1380000.0,4,3.25,4290,12103,1.0,0.0,3.0,...,11,2690,1600.0,1997,0.0,98006,47.5503,-122.102,3860,11244
314,4139480200,12/9/2014,1400000.0,4,3.25,4290,12103,1.0,0.0,3.0,...,11,2690,1600.0,1997,0.0,98006,47.5503,-122.102,3860,11244
324,7520000520,9/5/2014,232000.0,2,1.00,1240,12092,1.0,NaN,0.0,...,6,960,280.0,1922,1984.0,98146,47.4957,-122.352,1820,7460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20654,8564860270,3/30/2015,502000.0,4,2.50,2680,5539,2.0,NaN,0.0,...,8,2680,0.0,2013,0.0,98045,47.4759,-121.734,2680,5992
20763,6300000226,6/26/2014,240000.0,4,1.00,1200,2171,1.5,0.0,0.0,...,7,1200,0.0,1933,0.0,98133,47.7076,-122.342,1130,1598
20764,6300000226,5/4/2015,380000.0,4,1.00,1200,2171,1.5,0.0,0.0,...,7,1200,0.0,1933,0.0,98133,47.7076,-122.342,1130,1598
21564,7853420110,10/3/2014,594866.0,3,3.00,2780,6000,2.0,0.0,0.0,...,9,2780,0.0,2013,0.0,98065,47.5184,-121.886,2850,6000


In [5]:
df.drop_duplicates(subset='id', keep='first', inplace=True, ignore_index=True)

In [6]:
df[df.duplicated(subset='id', keep='first')==True]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


Drop unnecessary columns. Features that are time related ('date', 'yr_renovated', etc) are not useful to us because we can only build homes starting from tomorrow. Likewise, 'condition' and 'grade' will also be removed because we are only building new homes, and will do so at the highest quality that we can.

In [7]:
df.drop(columns=['id','date','condition','grade','yr_built','yr_renovated'], inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21420 entries, 0 to 21419
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21420 non-null  float64
 1   bedrooms       21420 non-null  int64  
 2   bathrooms      21420 non-null  float64
 3   sqft_living    21420 non-null  int64  
 4   sqft_lot       21420 non-null  int64  
 5   floors         21420 non-null  float64
 6   waterfront     19067 non-null  float64
 7   view           21357 non-null  float64
 8   sqft_above     21420 non-null  int64  
 9   sqft_basement  21420 non-null  object 
 10  zipcode        21420 non-null  int64  
 11  lat            21420 non-null  float64
 12  long           21420 non-null  float64
 13  sqft_living15  21420 non-null  int64  
 14  sqft_lot15     21420 non-null  int64  
dtypes: float64(7), int64(7), object(1)
memory usage: 2.5+ MB


Couldn't find a way to replace the null entries in 'view' and 'waterfront' columns since they are binary and there is no way of knowing what should be there. We will drop those rows, as it still leaves us ~90% of the data to work with.

In [9]:
df.dropna(subset=['view'], inplace=True)

In [10]:
df.dropna(subset=['waterfront'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19010 entries, 1 to 21419
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          19010 non-null  float64
 1   bedrooms       19010 non-null  int64  
 2   bathrooms      19010 non-null  float64
 3   sqft_living    19010 non-null  int64  
 4   sqft_lot       19010 non-null  int64  
 5   floors         19010 non-null  float64
 6   waterfront     19010 non-null  float64
 7   view           19010 non-null  float64
 8   sqft_above     19010 non-null  int64  
 9   sqft_basement  19010 non-null  object 
 10  zipcode        19010 non-null  int64  
 11  lat            19010 non-null  float64
 12  long           19010 non-null  float64
 13  sqft_living15  19010 non-null  int64  
 14  sqft_lot15     19010 non-null  int64  
dtypes: float64(7), int64(7), object(1)
memory usage: 2.3+ MB


Investigate 'sqft_basement column' because it should be a float but is currently an object.

In [11]:
df['sqft_basement'].value_counts()

0.0       11282
?           413
600.0       189
500.0       185
700.0       184
          ...  
935.0         1
506.0         1
2190.0        1
508.0         1
906.0         1
Name: sqft_basement, Length: 297, dtype: int64

Looks like there are some rows with '?' as entries. We will change these to '0.0' for now so we can convert the column to float.

In [12]:
df.loc[df['sqft_basement']=='?', ['sqft_basement']] = '0.0'
df['sqft_basement'].value_counts()

0.0       11695
600.0       189
500.0       185
700.0       184
800.0       171
          ...  
506.0         1
2360.0        1
784.0         1
2190.0        1
906.0         1
Name: sqft_basement, Length: 296, dtype: int64

In [13]:
df['sqft_basement'] = df['sqft_basement'].astype(float)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19010 entries, 1 to 21419
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          19010 non-null  float64
 1   bedrooms       19010 non-null  int64  
 2   bathrooms      19010 non-null  float64
 3   sqft_living    19010 non-null  int64  
 4   sqft_lot       19010 non-null  int64  
 5   floors         19010 non-null  float64
 6   waterfront     19010 non-null  float64
 7   view           19010 non-null  float64
 8   sqft_above     19010 non-null  int64  
 9   sqft_basement  19010 non-null  float64
 10  zipcode        19010 non-null  int64  
 11  lat            19010 non-null  float64
 12  long           19010 non-null  float64
 13  sqft_living15  19010 non-null  int64  
 14  sqft_lot15     19010 non-null  int64  
dtypes: float64(8), int64(7)
memory usage: 2.3 MB


We noticed that sqft_living = sqft_above + sqft_basement. Because these columns are correlated, we will get rid of 'sqft_above' and 'sqft_basement'. Instead, we will have a binary column 'basement' to show whether or not the house has a basement.

In [15]:
df['basement'] = np.where(df['sqft_basement'] > 0, 1, 0)
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,basement
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,2170,400.0,98125,47.7210,-122.319,1690,7639,1
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,770,0.0,98028,47.7379,-122.233,2720,8062,0
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,1050,910.0,98136,47.5208,-122.393,1360,5000,1
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,1680,0.0,98074,47.6168,-122.045,1800,7503,0
5,1230000.0,4,4.50,5420,101930,1.0,0.0,0.0,3890,1530.0,98053,47.6561,-122.005,4760,101930,1


In [16]:
df.drop(columns=['sqft_basement', 'sqft_above'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19010 entries, 1 to 21419
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          19010 non-null  float64
 1   bedrooms       19010 non-null  int64  
 2   bathrooms      19010 non-null  float64
 3   sqft_living    19010 non-null  int64  
 4   sqft_lot       19010 non-null  int64  
 5   floors         19010 non-null  float64
 6   waterfront     19010 non-null  float64
 7   view           19010 non-null  float64
 8   zipcode        19010 non-null  int64  
 9   lat            19010 non-null  float64
 10  long           19010 non-null  float64
 11  sqft_living15  19010 non-null  int64  
 12  sqft_lot15     19010 non-null  int64  
 13  basement       19010 non-null  int32  
dtypes: float64(7), int32(1), int64(6)
memory usage: 2.1 MB


Run baseline model with just scrubbed data. No improvements made yet.

In [25]:
x = df.drop(['price'], axis=1)
y = df['price']
regression = LinearRegression()
crossvalidation = KFold(n_splits = 10, shuffle = True, random_state = 1)
baseline_r2 = np.mean(cross_val_score(regression, x, y, scoring='r2', cv=crossvalidation))
baseline

0.6482664708838116

In [24]:
import statsmodels.api as sm
model = sm.OLS(y,x)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.888
Model:                            OLS   Adj. R-squared (uncentered):              0.888
Method:                 Least Squares   F-statistic:                          1.163e+04
Date:                Mon, 05 Jul 2021   Prob (F-statistic):                        0.00
Time:                        16:05:51   Log-Likelihood:                     -2.6078e+05
No. Observations:               19010   AIC:                                  5.216e+05
Df Residuals:                   18997   BIC:                                  5.217e+05
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -4.221e+04   2169.268    -19.457      0.000   -4.65e+04    -3.8e+04
bathrooms      1.461e+04   3543.910      4.122      0.000    7661.860    2.16e+04
sqft_living     250.5038      3.623     69.145      0.000     243.403     257.605
sqft_lot          0.1849      0.057      3.266      0.001       0.074       0.296
floors         -102.0256   3753.807     -0.027      0.978   -7459.820    7255.769
waterfront     5.982e+05   2.01e+04     29.769      0.000    5.59e+05    6.38e+05
view           6.795e+04   2425.238     28.020      0.000    6.32e+04    7.27e+04
zipcode        -853.1326     19.357    -44.074      0.000    -891.074    -815.192
lat            7.344e+05   1.21e+04     60.599      0.000    7.11e+05    7.58e+05
long           -3.99e+05   1.47e+04    -27.124      0.000   -4.28e+05    -3.7e+05
sqft_living15    50.8599      3.697     13.759      0.000      43.614      58.106
sqft_lot15       -0.4846      0.084     -5.741      0.000      -0.650      -0.319
basement      -1.725e+04   3821.621     -4.515      0.000   -2.47e+04   -9762.743
==============================================================================
Omnibus:                    14274.117   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           914343.419
Skew:                           3.043   Prob(JB):                         0.00
Kurtosis:                      36.426   Cond. No.                     1.27e+06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.27e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""